In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from adaptnn.model_fitting import NiruDataModel
torch.set_default_device('cuda:0')

In [2]:
model = NiruDataModel()
print(model.dataset.start_idx_X_train)
print(model.dataset.start_idx_X_train.shape)

Building multi-layer convolutional model for 14 neurons and image size 50 x 50
Adding Tucker convolutional layer of size (15, 15, 40) and 8 channels with factorization type spatial and rank 6.
Adding nonlinearity: Softplus.
Adding Tucker convolutional layer of size (11, 11, 12) and 8 channels with factorization type spatial and rank 6.
Adding nonlinearity: Softplus.
Adding final 3D batch normalization layer.
Adding full-connected linear layer: [8, 26, 26] to 14.
Adding output nonlinearity: Softplus.
Model initialized.
tensor([     0,    300,    600,  ..., 358800, 359100, 359400], device='cuda:0')
torch.Size([1199])


/home/latimerk/gitRepos/adaptnn/src/adaptnn/retina_datasets.py:101: UserWarning: Segmentation of dataset leaves 51 time bins unseen.
  warnings.warn(f"Segmentation of dataset leaves {unused_bins} time bins unseen.")


In [3]:
model.dataset.segment_length_bins = 500
X1,Y1 = model.dataset[0]
X1 = X1.unsqueeze(0)
Y1 = Y1.unsqueeze(0)
X2,Y2 = model.dataset[1]
X2 = X2.unsqueeze(0)
Y2 = Y2.unsqueeze(0)

X = torch.cat([X1,X2],dim=0)
Y = torch.cat([Y1,Y2],dim=0)
# X = X1
# Y = Y1

print(X.shape)
print(Y.shape)
criterion = model.get_loss_function()
optimizer = torch.optim.Adam(model.model.parameters())

/home/latimerk/gitRepos/adaptnn/src/adaptnn/retina_datasets.py:101: UserWarning: Segmentation of dataset leaves 251 time bins unseen.
  warnings.warn(f"Segmentation of dataset leaves {unused_bins} time bins unseen.")


torch.Size([2, 1, 50, 50, 550])
torch.Size([2, 14, 500])


In [4]:

optimizer.zero_grad()

#Forward to get outputs
%timeit  -r 5 -n 10  optimizer.zero_grad(); outputs=model.model(X)

outputs=model.model(X)

#calculate loss
%timeit  -r 5 -n 10  optimizer.zero_grad(); outputs=model.model(X); loss=criterion(outputs, Y) 

loss=criterion(outputs, Y) 

#getting gradients wrt parameters
%timeit  -r 5 -n 10  optimizer.zero_grad(); outputs=model.model(X); loss=criterion(outputs, Y); loss.backward()

The slowest run took 95.50 times longer than the fastest. This could mean that an intermediate result is being cached.
55.9 ms ± 69.1 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
The slowest run took 309.77 times longer than the fastest. This could mean that an intermediate result is being cached.
211 ms ± 262 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
1.12 s ± 247 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [ ]:
model.train(epochs=200,print_every=1,print_every_batch=50,
            optimizer_params = {"lr" : 1e-3},
            scheduler_params = {"start_factor" : 1.0, "end_factor" : 0.1, "total_iters" : 100},
            batch_params = {"batch_size":16, "shuffle":True},
            # penalty_params = {"en_lambda" : 0.0001}
            )

In [ ]:
print(model.model[4].weight_tucker.core.shape)
for ii in model.model[4].weight_tucker.factors:
    print(ii.shape)

In [ ]:
print(model.model[4].weight_tucker.core.numel())
tt = model.model[4].weight_tucker.core.numel()
for ii in model.model[4].weight_tucker.factors:
    print(ii.numel())
    tt += ii.numel()
print(tt)

In [ ]:
model.model[4]